# Deep Learning for Robotics Assignment 1 - Behavioral Cloning

This assignment will give you a tutorial on behavioral cloning, a very important tool for robot learning.  Behvioral cloning allows us to learn policies from datasets of previously collected demonstrations.  This is very powerful for a number of reasons.  First, these offline approaches do not require environment interactions (we do not need to query the environment for rewards as you need to do with Reinforcement Learning).  For robotics this is particularly helpful as environment interaction can be expensive or dangerous.  Even more importantly, offline approaches are necessary for robot learning as they allow for much greater scalability and portability (which is exactly what we are looking for in this era of deep learning).  Datasets of demonstrations can be shared across institutions and grown over time (think RT-X dataset).  These large scale datasets can help bring robotics closer to the successes of computer vision and NLP.  If you work in robotics you may realize that this is not exactly how things currently work, datasets are often not reused and it is very common to collect new data for every experiment.  This is exactly why we need better methods for behavioral cloning.  Learning effictive policies from diverse demonstration datasets is a very active area of research.

### Imports and Environment Setup

In [1]:
# mount your google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
%%capture
# install the necessary packages
!git clone https://github.com/ARISE-Initiative/robomimic
!pip install -e robomimic/
!git clone https://github.com/ARISE-Initiative/robosuite
!pip install mujoco
!pip install egl-probe


In [6]:
import os
import sys

# change path to your DLM folder
path_to_dlm = '/content/drive/My Drive/DLM'

sys.path.append(path_to_dlm)
sys.path.append('./robomimic/')
sys.path.append('./robosuite/')

%load_ext autoreload
%autoreload 2

### Understanding the Datasets

Behavioral cloning is supervised learning.  We take in a dataset of (state, action) pairs, or (observation, action) pairs if we do not have access to the full state space.  The goal of behavioral cloning is to train a neural network that learns the mapping from states to actions.  Because we are directly learning the action distribution from the dataset, behavioral cloning is also called imitation learning.  A key feature of behavioral cloning is that we do not have access to reward labels.  This is good because labels are expensive, but bad because it is hard to contend with suboptimal data.  This is why many methods will assume access to expert demonstrations.  However, developing methods that can learn from suboptimal policies is an active area of research.  In later parts of this assignment you will experiment with how suboptimallity can affect training performance.

For this assignment you will be working with the robomimic codebase and datasets.  This section goes over how datasets are stored in robomimic.  We will start by downloading a demonstration dataset for the square task.  This requires putting a square peg through a square hole.

In [7]:
import robomimic.utils.file_utils as FileUtils
from robomimic import DATASET_REGISTRY

# set download folder
download_folder = 'datasets/square/ph' #os.path.join(path_to_dlm, 'datasets/square/ph')
os.makedirs(download_folder, exist_ok=True)

#download the dataset
task = "square"
dataset_type = "ph"
hdf5_type = "low_dim"
FileUtils.download_url(
    url=DATASET_REGISTRY[task][dataset_type][hdf5_type]["url"],
    download_dir=download_folder,
)

# enforce that the dataset exists
dataset_path = os.path.join(download_folder, "low_dim_v141.hdf5")
assert os.path.exists(dataset_path)

ROBOMIMIC WARNING(
    No private macro file found!
    It is recommended to use a private macro file
    To setup, run: python /content/./robomimic/robomimic/scripts/setup_macros.py
)
yes


low_dim_v141.hdf5: 53.0MB [00:06, 7.87MB/s]                            


Next we look at the contents of the dataset.  There is a lot more information stored in these dataset, but you just need to worry about the actions and the observations.  Actions are commonly stored as the delta joint positions of the robot, and the joint torques are calculated using inverse kinematics (this is just handled by a pre-written controller).  Observations include proprioception (joint/end-effector positions and velocites) as well as images or low dimensional states.

In [8]:
import h5py
# open file
f = h5py.File(dataset_path, "r")

# each demonstration is a group under "data".  each demonstration is named "demo_#" where # is a number, starting from 0
demos = list(f["data"].keys())
num_demos = len(demos)

print("hdf5 file {} has {} demonstrations".format(dataset_path, num_demos))

# look at first demonstration
demo_key = demos[0]
demo_grp = f["data/{}".format(demo_key)]

# actions is a num numpy array of shape (time, action dim)
actions = demo_grp["actions"][:]
print("shape of actions {}".format(actions.shape))

#Each observation is a dictionary that maps modalities to numpy arrays of shape (time, obs modality dim)
print("observations:")
for obs, obs_key in demo_grp["obs"].items():
    print("{} - shape {}".format(obs , obs_key.shape))

hdf5 file /content/drive/My Drive/DLM/datasets/square/ph/low_dim_v141.hdf5 has 200 demonstrations
shape of actions (127, 7)
observations:
object - shape (127, 14)
robot0_eef_pos - shape (127, 3)
robot0_eef_quat - shape (127, 4)
robot0_eef_vel_ang - shape (127, 3)
robot0_eef_vel_lin - shape (127, 3)
robot0_gripper_qpos - shape (127, 2)
robot0_gripper_qvel - shape (127, 2)
robot0_joint_pos - shape (127, 7)
robot0_joint_pos_cos - shape (127, 7)
robot0_joint_pos_sin - shape (127, 7)
robot0_joint_vel - shape (127, 7)


For pick and place tasks (which is what we are concerned with in this assignment), it is common to just use end-effector 6-DoF position and gripper state for the state of the robot.  Additionally, to speed up computation time we will just be using low-dimensional states (not images) for the object observations.  While you are welcome to use additional observations as you run your experiments, lets set up the observation space as discribed above

In [9]:
import robomimic.utils.obs_utils as ObsUtils

obs_spec = dict(
    obs=dict(
            low_dim=[
                    "object",
                    "robot0_eef_pos",
                    "robot0_eef_quat",
                    "robot0_gripper_qpos",
                    ],
            rgb=[],
        ),
)
ObsUtils.initialize_obs_utils_with_obs_specs(obs_modality_specs=obs_spec)


============= Initialized Observation Utils with Obs Spec =============

using obs modality: low_dim with keys: ['robot0_gripper_qpos', 'robot0_eef_pos', 'robot0_eef_quat', 'object']
using obs modality: rgb with keys: []


In [10]:
# we can also grab multiple timesteps at once directly, or even the full trajectory at once
first_ten_actions = demo_grp["actions"][:10]
print("shape of first ten actions {}".format(first_ten_actions.shape))
all_actions = demo_grp["actions"][:]
print("shape of all actions {}".format(all_actions.shape))

shape of first ten actions (10, 7)
shape of all actions (127, 7)


Finally, lets playback some of the demonstrations so you can visualize exactly what these demonstrations look like.

In [ ]:
import helper
from IPython.display import Video
import robomimic.utils.obs_utils as ObsUtils

# prepare to write playback trajectories to video
video_path = os.path.join(download_folder, "playback.mp4")

helper.playback_demos(video_path, dataset_path, num_rollouts = 5)

# view the trajectories!
Video(video_path, embed=True)

# Implementing the Algorithms

As mentioned before, behavioral cloning is just supervised learning.  So common deep learning networks and techniques can easily be applied to imitation learning.  While in some cases learning the action distribution can be difficult, the main challenge with behavioral cloning (and with many supervised learning problems) is dealing with distribution shifts.  If the conditions during evaluation are not exactly the same as conditions during data collection, then networks often make inaccurate action predictions in these unseen states.  In sequential decision making scenarios (such as robotics), the challenges of out of distribution data are even more extreme.  This is because of compounding errors.  An inaccurate action prediction will throw you into an out of distribution state, and once you are out of distribution, subsequent inaccurate predictions take you even farther from the demonstration data.  In other supervised learning settings (such as classification for computer vision), an incorrect prediction is not going to make future predictions any less accurate.  So even in these simulation environments where the evaluation conditions are exactly the same as they were during data collection, you will notice that completing these tasks with a high success rate is not trivial.  Developing imitation learning methods that can better generalize to out of distribution scenarios is an important area of research.


You will be implerementing three behavioral cloning algorithms in this section.  For each algorithms do the following: 

**Instructions - Hit threshold for rollout success rate, plot training and validation loss curves**

## MLP (10 points)

First we will be implementing the most simple version of Behavioral Cloning - a Multi-Layer Perceptron that directly maps observations to actions.  For this part of the assignment, you will be filling in the MLP class.  

In [ ]:
# Load in the data for training
from helper import load_data_for_training
obs_keys = ["object", "robot0_eef_pos", "robot0_eef_quat", "robot0_gripper_qpos"]
seq_len = 1
batch_size = 100
train_loader, valid_loader = load_data_for_training(
    dataset_path=dataset_path,
    obs_keys=obs_keys,
    seq_len=seq_len,
    batch_size=batch_size
)

SequenceDataset: loading dataset into memory...
100%|██████████| 180/180 [00:00<00:00, 390.71it/s]
SequenceDataset: caching get_item calls...
100%|██████████| 27165/27165 [00:01<00:00, 18656.09it/s]
SequenceDataset: loading dataset into memory...
100%|██████████| 20/20 [00:00<00:00, 299.75it/s]
SequenceDataset: caching get_item calls...
100%|██████████| 2989/2989 [00:00<00:00, 21514.64it/s]
batch keys: dict_keys(['actions', 'obs'])
observation shapes: 
object shape: torch.Size([100, 1, 14])
robot0_eef_pos shape: torch.Size([100, 1, 3])
robot0_eef_quat shape: torch.Size([100, 1, 4])
robot0_gripper_qpos shape: torch.Size([100, 1, 2])
action shape: torch.Size([100, 1, 7])


While filling in the class you will need to do the following:
* Initialize the network in the init function
* Implement the logic for training on a batch of data
* Implement the logic for prediction an action based on an observation


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

class DLMMLP(nn.Module):
  def __init__(
      self,
      input_dim,
      hidden_dims,
      output_dim,
      obs_keys
    ):
    """
    Args:
    - input_dim (int): dimension of input obs
    - hidden_dims (list of int): dimensions of hidden layers
    - output_dim (int): dimension of output action
    - obs_keys (list of str): keys of obs modalities
    """
    super(DLMMLP, self).__init__()
    self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    ######################################
    # Define the model architecture here #
    ######################################
    self.hidden_layers = nn.ModuleList()
    previous_size = input_dim

    # Create hidden layers
    for hidden_dim in hidden_dims:
        self.hidden_layers.append(nn.Linear(previous_size, hidden_dim))
        self.hidden_layers.append(nn.ReLU())
        previous_size = hidden_dim

    # Output layer
    self.output_layer = nn.Linear(previous_size, output_dim)
    self.optimizer = optim.Adam(self.parameters(), lr=0.0001)
    ######################################
    # End of code                        #
    ######################################

    self.obs_keys = obs_keys
    self.to(self.device)
    self.epoch = 0

  def train_on_batch(self, batch, validate):
    """
    Args:
    - batch (dict): batch of data batch['obs'].shape = (B, T, Obs_dim) and batch['actions'].shape = (B, T, Action_dim)
    - validate (bool): whether batch is for validation

    Returns:
    - loss_val (float): value of loss
    """
    ######################################
    # Define the training loop here      #
    ######################################
    a_target = batch['actions'][:, 0, :]
    actions = self.forward(batch['obs'])[:, 0, :]
    loss = nn.MSELoss()(actions, a_target)
    if not validate:
      self.optimizer.zero_grad()
      loss.backward()
      self.optimizer.step()
    loss_val = loss.item()
    ######################################
    # End of code                        #
    ######################################
    return loss_val

  def forward(self, x):
    """
    You do not actually have to use this method, 
    the environment will call get_action directly.
    But it may be useful to define the forward pass here.
    """
    ######################################
    # Define the forward pass here       #
    ######################################
    # Forward pass through hidden layers
    for layer in self.hidden_layers:
        x = layer(x)

    # Output layer
    x = self.output_layer(x)
    ######################################
    # End of code                        #
    ######################################


    return x

  def get_action(self, obs):
    """
    Args:
    - obs (dict): dictionary of observations
    Returns:
    - action (np.ndarray): action
    """
    ######################################
    # Define the forward pass here       #
    ######################################
    self.eval()
    with torch.no_grad():
      action = self.forward(obs)
    return action
    ######################################
    # End of code                        #
    ######################################

  def reset(self):
    return

  def save(self, path):
    torch.save(self.state_dict(), path)

  def load(self, path):
    #get epoch from end of path
    self.epoch = int(path.split("_")[-1].split(".")[0])
    self.load_state_dict(torch.load(path))

  def set_eval(self):
    self.eval()


Create the model and train.  Do not worry about having a complicated network architecture, even a simple MLP will be able to model the observation-action distribution fairly well.  But hopefully you can see there are a lot of parameters you could change during your experiments at the end of this assignment.

In [ ]:
model = DLMMLP(23, [1024, 1024], 7, obs_keys = obs_keys)

Now it is time to train the model, you should be able to achieve the target success rate after ~500 training epochs.

**Target Success Rate: 60%**

In [ ]:
from helper import train

save_path = os.path.join(path_to_dlm, "trainings/mlp")
if not os.path.exists(save_path):
  os.makedirs(save_path)

# train_losses and valid losses are lists of (loss, epoch) tupples
train_losses, valid_losses = train(model, train_loader, valid_loader, num_epochs=100, save_path=save_path)

Epoch: 0 Train Loss: 0.13976469729095697 Valid Loss: 0.11293840631842614
Epoch: 10 Train Loss: 0.04677587212063372 Valid Loss: 0.05244902595877647
Epoch: 20 Train Loss: 0.04051734459832968 Valid Loss: 0.04755133421470722
Epoch: 30 Train Loss: 0.037218229735598844 Valid Loss: 0.04930283203721046
Epoch: 40 Train Loss: 0.03510382235981524 Valid Loss: 0.042729096673429015
Epoch: 50 Train Loss: 0.03414715225945281 Valid Loss: 0.04736744426190853
Epoch: 60 Train Loss: 0.03292228382162969 Valid Loss: 0.04489025467385848
Epoch: 70 Train Loss: 0.032073177862912416 Valid Loss: 0.04358015321195126
Epoch: 80 Train Loss: 0.031641655930263156 Valid Loss: 0.04033401881655057
Epoch: 90 Train Loss: 0.031027718152686515 Valid Loss: 0.04031900819391012


In [ ]:
# Load saved model if you have checkpoints from previous training runs
# model.load(os.path.join(save_path, "model_100.pth"))

It will take quite a while to run the rollout loop, so start by visualizing one or two rollouts.  Does the robots actions look reasonable?  Or is it bouncing around wildly?  Remember you have visualizations of the original demonstrations earlier in this notebook if you want to remember the behavior we are trying to imitate.  Once you are confident that your model is reasonable, then you can take the time to run the full 50 rollouts.

**Do not delete the final output of this cell!  We need to see the final success rate to score you correctly.**

In [ ]:
from helper import rollout
import imageio

# create a video writer
video_path = "rollout.mp4"
video_writer = imageio.get_writer(video_path, fps=20)

# you can change this while debugging, but final success rate should be over 50 rollouts
num_rollouts = 50

success_rate = rollout(model,
                      dataset_path,
                      horizon = 400,
                      video_writer = video_writer,
                      obs_keys = obs_keys,
                      num_rollouts = num_rollouts)
print("Success rate over {} rollouts: {}".format(num_rollouts, success_rate))

video_writer.close()

[robosuite WARNING] No private macro file found! (macros.py:53)
[robosuite WARNING] It is recommended to use a private macro file (macros.py:54)
[robosuite WARNING] To setup, run: python /content/./robosuite/robosuite/scripts/setup_macros.py (macros.py:55)


Created environment with name NutAssemblySquare
Action size is 7
Success rate over 50 rollouts: 0.58


In [ ]:
from IPython.display import Video
Video(video_path, embed=True)

Output hidden; open in https://colab.research.google.com to view.

**TODO:**  Plot your training and validation loss curves.  Indicate the checkpoint epoch you used to generate the success rate above.  Write 2/3 sentences reflecting on your takeaways from this process.

### RNN (10 points)

Because robotics is all about sequential decision making, it makes sense to use a sequence-to-sequence model for behavioral cloning.  In this section you will be implementing an RNN for action prediction.
It is also common to see transformers used for action prediction. We will now reload the dataset with a sequence length of 10.  You can change this sequence length in your experiments, but for now please use 10.

In [11]:
from helper import load_data_for_training
# For experiments, it is probably best to just alter these parameters
obs_keys = ["object", "robot0_eef_pos", "robot0_eef_quat", "robot0_gripper_qpos"]
seq_len = 10
batch_size = 100
train_loader, valid_loader = load_data_for_training(
    dataset_path=dataset_path,
    obs_keys=obs_keys,
    seq_len=seq_len,
    batch_size=batch_size
)

SequenceDataset: loading dataset into memory...
100%|██████████| 180/180 [00:00<00:00, 427.41it/s]
SequenceDataset: caching get_item calls...
100%|██████████| 27165/27165 [00:01<00:00, 14441.56it/s]
SequenceDataset: loading dataset into memory...
100%|██████████| 20/20 [00:00<00:00, 252.17it/s]
SequenceDataset: caching get_item calls...
100%|██████████| 2989/2989 [00:00<00:00, 12675.60it/s]
batch keys: dict_keys(['actions', 'obs'])
observation shapes: 
object shape: torch.Size([100, 10, 14])
robot0_eef_pos shape: torch.Size([100, 10, 3])
robot0_eef_quat shape: torch.Size([100, 10, 4])
robot0_gripper_qpos shape: torch.Size([100, 10, 2])
action shape: torch.Size([100, 10, 7])


Below you will implement the RNN, here are the guidlines:
* Use nn.LSTM
* Inilitalize hidden state to zeros
* Have a large hidden state size and use a fully connected layer to process the outputs into the correct action dimension
* For generating actions, keep track of the hidden state and re-initialize after self.hidden_state_horizon action steps

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import robomimic.utils.tensor_utils as TensorUtils

class DLM_RNN(nn.Module):
  def __init__(
      self,
      input_dim,
      hidden_dim,
      num_layers,
      output_dim,
      obs_keys,
      rnn_horizon
    ):
    """
    Args:
    - input_dim (int): dimension of input obs
    - hidden_dim (int): dimension of hidden state
    - num_layers (int): number of hidden layers
    - output_dim (int): dimension of output action
    - obs_keys (list of str): keys of obs modalities
    - rnn_horizon (int): how many steps to run the RNN before resetting the hidden state
    """
    super(DLM_RNN, self).__init__()
    self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    self.obs_keys = obs_keys

    ######################################
    # Define the model architecture here #
    # Also initialize parameters         #
    ######################################

    # use nn.LSTM
    self.rnn_net = nn.LSTM(
        input_size=input_dim,
        hidden_size=hidden_dim,
        num_layers=num_layers,
        batch_first=True
    )

    self.hidden_size = hidden_dim
    self.num_layers = num_layers
    self.hidden_state = None
    self.hidden_state_horizon = rnn_horizon
    self.hidden_state_counter = 0

    # Output layer
    self.output_layer = nn.Linear(hidden_dim, output_dim)
    self.optimizer = optim.Adam(self.parameters(), lr=0.0001)

    ######################################
    # End of code                        #
    ######################################
    self.to(self.device)
    self.epoch = 0

  def train_on_batch(self, batch, validate):
    """
    Args:
    - batch (dict): batch of data batch['obs'].shape = (B, T, Obs_dim) and batch['actions'].shape = (B, T, Action_dim)
    - validate (bool): whether batch is for validation

    Returns:
    - loss_val (float): value of loss
    """
    ######################################
    # Define the training loop here      #
    ######################################
    a_target = batch['actions']
    actions, _ = self.forward(batch['obs'])
    loss = nn.MSELoss()(actions, a_target)
    if not validate:
      self.optimizer.zero_grad()
      loss.backward()
      self.optimizer.step()
    loss_val = loss.item()
    del loss
    ######################################
    # End of code                        #
    ######################################
    return loss_val

  def initialize_hidden_state(self, batch_size):
    """
    Args:
    - batch_size (int): size of batch
    Returns:
    - (h_0, c_0) : initial hidden state and cell state for LSTM
    """
    ######################################
    # Define the initial LSTM state      #
    ######################################
    h_0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(self.device)
    c_0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(self.device)
    ######################################
    # End of code                        #
    ######################################
    return h_0, c_0

  def forward(self, x, hidden_state = None):
    """
    Args:
    - x (torch.Tensor): input obs of shape (B, T, Obs_dim)
    - hidden_state (tuple): None or hidden state and cell state for LSTM
    Returns:
    - output (torch.Tensor): output action of shape (B, T, Action_dim)
    - hidden_state (tuple): hidden state and cell state for LSTM
    """
    #######################################################
    # Define the forward pass here                        #
    # Again, you do not actually have to use this method, #
    # the environment will call get_action directly.      #
    #######################################################
    # Forward pass through hidden layers
    if hidden_state is None:
      hidden_state = self.initialize_hidden_state(x.shape[0])
    x, hidden_state = self.rnn_net(x, hidden_state)
    # Forward pass through output layer
    action = self.output_layer(x)
    #######################################################
    # End of code                                         #
    #######################################################
    return action, hidden_state

  def get_action(self, obs):
    """
    Args:
    - obs (dict): dictionary of observations of shape (B, 1, Obs_dim)
    Returns:
    - action (np.ndarray): action of shape (B, action_dim)
    """
    #########################################
    # Implement logic for action prediction #
    #########################################
    self.eval()
    if self.hidden_state is None or self.hidden_state_counter % self.hidden_state_horizon == 0:
      self.hidden_state = self.initialize_hidden_state(obs.shape[0])
    self.hidden_state_counter += 1
    with torch.no_grad():
      action, self.hidden_state = self.forward(obs, self.hidden_state)
    action = action[:, 0]
    #########################################
    # End of code                           #
    #########################################
    return action

  def reset(self):
    #########################################
    # Implement reset logic here            #
    # Called at the start of each rollout   #
    # Maybe do something with hidden_state  #
    #########################################
    self.hidden_state = None
    self.hidden_state_counter = 0
    #########################################
    # End of code                           #
    #########################################
    return

  def save(self, path):
    torch.save(self.state_dict(), path)

  def load(self, path):
    #get epoch from end of path
    self.epoch = int(path.split("_")[-1].split(".")[0])
    self.load_state_dict(torch.load(path))


  def set_eval(self):
    self.eval()


Create the model and train.  Most of the challenge with this model is correctly handeling the hidden state.  

In [13]:
from helper import train
save_path = os.path.join(path_to_dlm, "trainings/rnn")
if not os.path.exists(save_path):
  os.makedirs(save_path)
model = DLM_RNN(23, 400, 2, 7, obs_keys, 10)


Now it is time to train the model, you should be able to achieve the target success rate after ~600 training epochs.

**Target Success Rate: 80%**

In [14]:
train_losses, valid_losses = train(model, train_loader, valid_loader, num_epochs=100, save_path=save_path)

Epoch: 0 Train Loss: 0.16887076555148645 Valid Loss: 0.13190208251277605
Epoch: 10 Train Loss: 0.05628627214087721 Valid Loss: 0.06189353714386622
Epoch: 20 Train Loss: 0.04748536379295675 Valid Loss: 0.05637195259332657
Epoch: 30 Train Loss: 0.043870016151820034 Valid Loss: 0.05431951247155666
Epoch: 40 Train Loss: 0.04090657289249494 Valid Loss: 0.05107941602667173
Epoch: 50 Train Loss: 0.03843427969909766 Valid Loss: 0.0472225159406662
Epoch: 60 Train Loss: 0.036898543147425 Valid Loss: 0.04592286025484403
Epoch: 70 Train Loss: 0.03530097229625372 Valid Loss: 0.04390392452478409
Epoch: 80 Train Loss: 0.033920120632768995 Valid Loss: 0.04260023062427839
Epoch: 90 Train Loss: 0.03252796465120114 Valid Loss: 0.04133480327824752


In [ ]:
# Load saved model if you have checkpoints from previous training runs
# model.load(os.path.join(save_path, "model_100.pth"))

As mentioned before, it will take quite a while to run the rollout loop, so start by visualizing one or two rollouts.  Does the robots actions look reasonable?  Or is it bouncing around wildly?  Remember you have visualizations of the original demonstrations earlier in this notebook if you want to remember the behavior we are trying to imitate.  Once you are confident that your model is reasonable, then you can take the time to run the full 50 rollouts.

**Do not delete the final output of this cell! We need to see your success rate to score you correctly**

In [ ]:
from helper import rollout
import imageio

# create a video writer
video_path = "rollout.mp4"
video_writer = imageio.get_writer(video_path, fps=20)

total_success_rate = 0
num_rollouts = 50
success_rate = rollout(model,
                      dataset_path,
                      horizon = 400,
                      video_writer = None,
                      obs_keys = obs_keys,
                      num_rollouts = num_rollouts)
print("Success rate over {} rollouts: {}".format(num_rollouts, success_rate))

video_writer.close()

[robosuite WARNING] No private macro file found! (macros.py:53)
[robosuite WARNING] It is recommended to use a private macro file (macros.py:54)
[robosuite WARNING] To setup, run: python /content/./robosuite/robosuite/scripts/setup_macros.py (macros.py:55)


Created environment with name NutAssemblySquare
Action size is 7
Success rate over 50 rollouts: 0.0


In [ ]:
Video(video_path, embed=True)

**TODO:**  Plot your training and validation loss curves.  Indicate the checkpoint epoch you used to generate the success rate above.  Write 2/3 sentences reflecting on your takeaways from this process.

### Diffusion Policy (10 points)

Now you will be implementing action prediction with a Diffusion Model.  Diffusion Models are an extremely popular class of generative models, especially because of their ability to represent multimodal distributions.  For this model your noise will be the shape of the actions that you want to predict and your conditioning information will be the observations.  If you need a refresher on Diffusion Models there are a lot of good resources online, but here are the papers that you are expected to understand to complete this part of the assignment.  You will have a hard time completing this assignment if you have never read these papers:

https://arxiv.org/pdf/2006.11239.pdf (Original Diffusion Model Paper)

https://arxiv.org/pdf/2303.04137.pdf (Diffusion Policy, the paper this implementation is based on)

In [ ]:
%%capture
!pip install diffusers

In [ ]:
from helper import load_data_for_training
# For experiments, it is probably best to just alter these parameters
obs_keys = ["object", "robot0_eef_pos", "robot0_eef_quat", "robot0_gripper_qpos"]
seq_len = 16
batch_size = 100
train_loader, valid_loader = load_data_for_training(
    dataset_path=dataset_path,
    obs_keys=obs_keys,
    seq_len=seq_len,
    batch_size=batch_size,
    frame_stack = 1,
)

SequenceDataset: loading dataset into memory...
100%|██████████| 180/180 [00:00<00:00, 195.03it/s]
SequenceDataset: caching get_item calls...
100%|██████████| 27165/27165 [00:05<00:00, 4947.64it/s]
SequenceDataset: loading dataset into memory...
100%|██████████| 20/20 [00:00<00:00, 221.45it/s]
SequenceDataset: caching get_item calls...
100%|██████████| 2989/2989 [00:00<00:00, 8482.27it/s]
batch keys: dict_keys(['actions', 'obs'])
observation shapes: 
object shape: torch.Size([100, 16, 14])
robot0_eef_pos shape: torch.Size([100, 16, 3])
robot0_eef_quat shape: torch.Size([100, 16, 4])
robot0_gripper_qpos shape: torch.Size([100, 16, 2])
action shape: torch.Size([100, 16, 7])


You will be implementing a very basic version of diffusion policy.  We will use a fixed beta schedule, a constant learning rate, no observation normalization, delta actions instead of absolute, and no EMA model.  For this reason your success rate may not be amazing, but this should give you a sense of how diffusion models can be used for action generation.  Feel free to add any of these components during your experiments if you want to get better success rates.  The guidelines for implementing this diffusion model policy are as follows:
* Use DDPMScheduler to add noise to actions during training
* Train noise_pred_net on these noised actions
* Use DDPMScheduler to remove noise during inference
* Use noise_pred_net to predict the amount of noise to remove at each timestep during inference
* You should be using 0 mean, 1 var gaussian noise (torch.randn)



In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import robomimic.utils.tensor_utils as TensorUtils
from copy import deepcopy

# do not import any additional diffusion model packages for this part of the assignment
# you are free to implement any packages you want for the experiments at the end of the assignment
from diffusers import DDPMScheduler
from helperModels import ConditionalUnet1D
from diffusers.training_utils import EMAModel



class DLM_Diffusion(nn.Module):
  def __init__(
      self,
      action_dim,
      obs_dim,
      denoising_steps,
      obs_horizon,
      prediction_horizon,
      action_horizon,
    ):

    super(DLM_Diffusion, self).__init__()
    self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    self.denoising_steps = denoising_steps
    self.action_dim = action_dim
    self.obs_horizon = obs_horizon
    self.prediction_horizon = prediction_horizon
    self.action_horizon = action_horizon
    self.obs_dim = obs_dim
    self.epoch = 0

    ######################################
    # Initialize the noise pred net      #
    ######################################
    # Input_dim is action dim
    # global con dim is flattened obs dim
    self.noise_pred_net = ConditionalUnet1D(
      input_dim=action_dim,
      global_cond_dim=obs_horizon * obs_dim,
    )
    ######################################
    # End of code                        #
    ######################################

    
    self.scheduler = DDPMScheduler(
        num_train_timesteps=self.denoising_steps,
        # the choise of beta schedule has big impact on performance
        # we found squared cosine works the best
        beta_schedule='squaredcos_cap_v2',
        # clip output to [-1,1] to improve stability
        clip_sample=True,
        # our network predicts noise (instead of denoised action)
        prediction_type='epsilon'
    )

    self.noise_pred_net.to(self.device)
    self.optimizer = optim.Adam(self.noise_pred_net.parameters(), lr=1e-4)
    self.ema = EMAModel(parameters=self.noise_pred_net.parameters(),power=0.75)
    self.ema.to(self.device)

  def train_on_batch(self, batch, validate):
    """
    Args:
    - batch (dict): batch of data batch['obs'].shape = (B, T, Obs_dim) and batch['actions'].shape = (B, T, Action_dim)
    - validate (bool): whether batch is for validation
    Returns:
    - loss_val (float): value of loss
    """
    ######################################
    # Define the training loop here      #
    ######################################
    # make sure to flatten obs (B, T, D) -> (B, --)
    a_target = batch['actions'].to(self.device)
    cond = batch['obs'].to(self.device)
    cond = cond[:, :self.obs_horizon]

    # flatten cond
    cond = cond.flatten(start_dim = 1)

    noise = torch.randn(tuple(a_target.shape), device = self.device)

    B = a_target.shape[0]

    timesteps = torch.randint(0, self.denoising_steps, (B,), device = self.device)

    noisy_data = self.scheduler.add_noise(a_target, noise, timesteps)

    noise_pred = self.noise_pred_net(noisy_data, timesteps, cond)

    loss = torch.nn.functional.mse_loss(noise_pred, noise)
    if not validate:
      self.optimizer.zero_grad()
      loss.backward()
      self.optimizer.step()
      self.ema.step(self.noise_pred_net.parameters())

    loss_val = loss.item()
    ######################################
    # End of code                        #
    ######################################
    return loss_val

  def get_action(self, obs):
    """
    Args:
    - obs (dict): dictionary of observations of shape (B, prediction_horizon, Obs_dim)
    Returns:
    - actions (np.ndarray): action of shape (B, action_horizon, action_dim)
    """
    # obs are (B, T, D) - make sure to flatten obs
    # also, remember obs_horizon is not the same as prediction_horizon
    original_params = deepcopy(self.noise_pred_net.state_dict())
    self.ema.copy_to(self.noise_pred_net.parameters())
    self.noise_pred_net.eval()
    
    ######################################################
    # Generate an action sample from the diffusion model #
    ######################################################
    obs = obs.to(self.device)
    obs = obs.flatten(start_dim = 1)
    sample = None
    with torch.no_grad():
        desired_shape = (obs.shape[0],) + (self.prediction_horizon, self.action_dim)

        sample = torch.randn(desired_shape, device = self.device)

        for t in range(self.denoising_steps -1, -1, -1):
            # convert t to tensor
            t = torch.ones(1, device = self.device).long() * t
            conditional_prediction = self.noise_pred_net(sample, t, obs)

            # make sure the students are passing in the seed to the denoise_step function
            sample = self.scheduler.step(conditional_prediction, t, sample).prev_sample
    # return the first self.action_horizon actions
    self.noise_pred_net.load_state_dict(original_params)
    # self.ema.copy_to(self.noise_pred_net.parameters())
    actions = sample[:, self.obs_horizon - 1 : self.obs_horizon -1 + self.action_horizon]
    ####################################################################################
    #                                 END OF YOUR CODE                                 #
    ####################################################################################
    return actions

  def reset(self):
    return

  def save(self, path):
    torch.save(self.state_dict(), path)
    # save the ema model, put ema before the .pth
    torch.save(self.ema.state_dict(), path.replace(".pth", "ema.pth"))

  def load(self, path):
    #get epoch from end of path
    self.epoch = int(path.split("_")[-1].split(".")[0])
    self.load_state_dict(torch.load(path))
    self.ema.load_state_dict(torch.load(path.replace(".pth", "ema.pth")))

  def set_eval(self):
    self.eval()

Create the mode and train.  It is really important that you understand the diffusion model training and inference process, if you are having troubles please review the basics of diffusion models.

In [ ]:
from helper import train
save_path = os.path.join(path_to_dlm, "trainings/diffusion")
if not os.path.exists(save_path):
  os.makedirs(save_path)

model = DLM_Diffusion(action_dim =7 , obs_dim = 23, denoising_steps=100, prediction_horizon=16, action_horizon=8, obs_horizon = 2)

number of parameters: 6.587828e+07


Now it is time to train the model, you should be able to achieve the target success rate after ~100 training epochs.

**Target Success Rate: 70%**

In [ ]:
training_losses, validation_losses = train(model, train_loader, valid_loader, num_epochs=50, save_path = save_path)

This will take a really really long time to run (even longer than the last ones).  Please keep this in mind when doing your experiments.  We do not recommend doing experiments that will increase you inference time, maybe forcus on experiments that lower inference time.

In [ ]:
from helper import rollout
import imageio

# create a video writer
video_path = "rollout.mp4"
video_writer = imageio.get_writer(video_path, fps=20)

total_success_rate = 0
num_rollouts = 50
success_rate = rollout(model,
                      dataset_path,
                      horizon = 400,
                      video_writer = video_writer,
                      obs_keys = obs_keys,
                      num_rollouts = num_rollouts,
                      obs_len = 2)
print("Success rate over {} rollouts: {}".format(num_rollouts, success_rate))

video_writer.close()

Created environment with name NutAssemblySquare
Action size is 7
ROBOMIMIC WARNING(
    Dataset and installed environment version mismatch!
    Dataset environment version: 1.4.1
    Installed environment version: 1.4.0
)
Success rate over 2 rollouts: 0.0


In [ ]:
from IPython.display import Video
Video(video_path, embed=True)

**TODO:**  Plot your training and validation loss curves.  Indicate the checkpoint epoch you used to generate the success rate above.  Write 2/3 sentences reflecting on your takeaways from this process.

### Experiments (30 points)

Okay now it is time to do some experiments with the models you implemented.  There are 4 different variables that we want you to investigate.  For each section you need to come up with a hypothesis, run an experiment to test that hypothesis, and then analyze whether or not your hypothesis was proved or disproved.  These do not need to be novel discoveries, we just want you to see how common dataset characteristics are impacted by different aspects of these models.  For each section below the points breakdown is as follows:

* (2pt) **Hypothesis** - Test how one aspect of the models you implemented effect the dataset variable
* (4pt) **Experiment** -  Discribe the experiment you ran and include at least one figure.
* (4pt) **Analysis** - Explain whether or not your hypothesis was proved or disproved.  What would be an interesting next experiment after seeing these results?


**To get full credit**, you must experiment with the parameters off all implemented models at least once.  For example, you can't just modify the parameters of the MLP for all experiments.  Failure to do so will results in -5 points.

**Task Difficulty** - Lift (easy) vs. Square (medium) vs. Tool Hang (Hard)

The robomimic demonstration datasets contain a number of different tasks with varying levels of difficulty.  Below we provide download links to three such tasks.  You can load them into the data loaders from previous sections as you perform your experiments.  

In [ ]:
# Download demonstration dataset
import robomimic.utils.file_utils as FileUtils

# the dataset registry can be found at robomimic/__init__.py
from robomimic import DATASET_REGISTRY

# set download folder for EASY task
lift_folder = os.path.join(path_to_dlm, 'datasets/lift/ph')
os.makedirs(lift_folder, exist_ok=True)

#download the dataset
task = "lift"
dataset_type = "ph"
hdf5_type = "low_dim"
FileUtils.download_url(
    url=DATASET_REGISTRY[task][dataset_type][hdf5_type]["url"],
    download_dir=lift_folder,
)

# enforce that the dataset exists
lift_path = os.path.join(lift_folder, "low_dim_v141.hdf5")
assert os.path.exists(dataset_path)

# set download folder for MEDIUM task
square_folder = os.path.join(path_to_dlm, 'datasets/square/ph')
os.makedirs(square_folder, exist_ok=True)

#download the dataset
task = "square"
dataset_type = "ph"
hdf5_type = "low_dim"
FileUtils.download_url(
    url=DATASET_REGISTRY[task][dataset_type][hdf5_type]["url"],
    download_dir=square_folder,
)

# enforce that the dataset exists
square_path = os.path.join(square_folder, "low_dim_v141.hdf5")
assert os.path.exists(square_path)

# set download folder for HARD task
tool_hang_folder = os.path.join(path_to_dlm, 'datasets/tool_hang/ph')
os.makedirs(square_folder, exist_ok=True)

#download the dataset
task = "tool_hang"
dataset_type = "ph"
hdf5_type = "low_dim"
FileUtils.download_url(
    url=DATASET_REGISTRY[task][dataset_type][hdf5_type]["url"],
    download_dir=tool_hang_folder,
)

# enforce that the dataset exists
tool_hang_path = os.path.join(tool_hang_folder, "low_dim_v141.hdf5")
assert os.path.exists(tool_hang_path)

Example Hypothesis: If I increase the depth of the MLP, I will see increased performance on harder tasks.

**Multi-Modality**

Multi-modality means a lot of things in robotics, but in this context we mean the dataset demonstrates multiple ways to reach any given goal.  This is a common characteristic of human demonstration datasets as different people will do the same task in different ways (and even the same person will have some variation in their demonstrations).  In this dataset the notion of multi-modality is exaggerated.  The task involves an agent picking up one of two blocks, and the

Example Hypothesis: Diffusion models will exhibit more multi-modality than RNNs.  RNNs will exhibit more multi-modality than MLPs.

**Dataset Quality**

In addition to different tasks, the robomimic datasets also have demonstrations from different quality demonstrators.  These were captures by either only proficient humans (ph) or a mixtured of human demonstrators (mh), including some suboptimal demonstrations.  Below we will download the mh dataset for the square task (ph dataset already downloaded in previous sections), but feel free to change this code and experiment with mh/ph on any task.


In [ ]:
# set download folder for MEDIUM task
square_mh_folder = os.path.join(path_to_dlm, 'datasets/square/mh')
os.makedirs(square_folder, exist_ok=True)

#download the dataset
task = "square"
dataset_type = "mh"
hdf5_type = "low_dim"
FileUtils.download_url(
    url=DATASET_REGISTRY[task][dataset_type][hdf5_type]["url"],
    download_dir=square_mh_folder,
)

# enforce that the dataset exists
square_mh_path = os.path.join(square_mh_folder, "low_dim_v141.hdf5")
assert os.path.exists(square_mh_path)



Example Hypothesis:  If I lower the diffusion model denoising steps, I will get worse performance on suboptimal data.

**Sequence Length/Prediction Horizon** -



This is not a dataset characteristic, but we want to make sure that you also experiment with the precition horizons and sequence lenghts of the models.  If you choose to do this for RNN, you will just be changing with the sequence length that you pass into the data loader.  If you choose to do this for the diffusion model you should experiment with the predicion horizon and the action horizon (make sure to read the diffusion policy paper if you are unsure what this means).

In [ ]:
from helper import load_data_for_training
obs_keys = ["object", "robot0_eef_pos", "robot0_eef_quat", "robot0_gripper_qpos"]
# you can alter the sequence length for RNN
seq_len = 16
batch_size = 100
train_loader, valid_loader = load_data_for_training(
    dataset_path=dataset_path,
    obs_keys=obs_keys,
    seq_len=seq_len,
    batch_size=batch_size,
)

# and the prediction horizon and action horizon for diffusion model
# note that the seq_len of data loader should match the prediction_horizon
prediction_horizon = 16
action_horizon = 8
model = DLM_Diffusion(
    action_dim =7 ,
    obs_dim = 23,
    denoising_steps=100,
    prediction_horizon=prediction_horizon,
    action_horizon=action_horizon
    )


Example hypothesis: If I increase the sequence length for the RNN, I will see improved task performance on hard tasks.  Additionally, increasing the sequence length will have no effect on the performance of easy tasks.